In [2]:
import numpy as np
import os

# vivarium core imports
from vivarium.core.experiment import pp
from vivarium.core.process import Generator
from vivarium.core.composition import (
    simulate_compartment_in_experiment,
    COMPARTMENT_OUT_DIR,
)
from vivarium.plots.simulation_output import plot_simulation_output
from vivarium.library.dict_utils import deep_merge

# import processes
from vivarium_bioscrape.composites.composite_general import insert_topology
from vivarium_bioscrape.processes.bioscrape import Bioscrape, get_model_species_ids
from vivarium_bioscrape.processes.one_way_map import OneWayMap
from vivarium_bioscrape.library.schema import array_from, array_to

ModuleNotFoundError: No module named 'vivarium_bioscrape.schema'

In [ ]:
'''Simulate the composite and plot results.'''

# make an output directory to save plots
out_dir = os.path.join(COMPARTMENT_OUT_DIR, NAME)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

model1_keys = get_model_species_ids('Notebooks/model1.xml')
model3_keys = get_model_species_ids('Notebooks/model3.xml')

# define the projection
model1_vector = np.array(['rna_T' == key for key in model1_keys])
model3_vector = np.array(['rna_T' == key for key in model3_keys])
projection_1_3 = np.outer(
    model1_vector/np.sum(model1_vector),
    model3_vector/np.sum(model3_vector))
# projection_3_1 = np.outer(
#     model3_vector/np.sum(model3_vector),
#     model1_vector/np.sum(model1_vector))
# TODO (William) HW -- is this needed?
model3_vector2 = np.array(['rna' in key for key in model3_keys])
projection_3_1 = np.outer(
    model3_vector2/np.sum(model3_vector2),
    model1_vector/np.sum(model1_vector))

# define map function
def map_1_3(states):
    input_array = array_from(states['source_deltas'])
    output_array = np.dot(input_array, projection_1_3)
    return array_to(model3_keys, output_array)

def map_3_1(states):
    input_array = array_from(states['source_deltas'])
    output_array = np.dot(input_array, projection_3_1)
    return array_to(model1_keys, output_array)


# configuration
time_step = 10
models = {
    '1': {
        'time_step': time_step,
        'sbml_file': 'Notebooks/model1.xml'},
    '3': {
        'time_step': time_step,
        'sbml_file': 'Notebooks/model3.xml'},
}
connections = [
    {'source': '1', 'target': '3', 'map': map_1_3},
    {'source': '3', 'target': '1', 'map': map_3_1}
]

# make the composite
composite = BioscrapeConnector(
    models=models,
    connections=connections,
)

# get processes and topology, print
network = composite.generate()
processes = network['processes']
topology = network['topology']

print('PROCESSES:')
pp(processes)
print('TOPOLOGY:')
pp(topology)

## Run a simulation
# initial state
initial_state = composite.initial_state()
initial_state['1_species']['dna_G'] = 1
initial_state['1_species']['rna_T'] = 10

# run a simulation
sim_settings = {
    'total_time': 1000,
    'initial_state': initial_state}
output = simulate_compartment_in_experiment(composite, sim_settings)

# plot simulation output
plot_settings = {}
plot_simulation_output(output, plot_settings, out_dir)